In [1]:
import pandas as pd 
import numpy as np
import spacy
from spacy.language import Language
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

In [2]:
data = pd.read_csv('preprocessed_labelled_data.csv')

In [3]:
data.head()

,NER,normalized_combined
0,"['bite size shredded rice biscuits', 'vanilla'...",no-bake nut cookies 1 cup firmly packed brown ...
1,"['cream of mushroom soup', 'beef', 'sour cream...",jewell ball s chicken 1 small jar chipped beef...
2,"['frozen corn', 'pepper', 'cream cheese', 'gar...",creamy corn 2 16 ounce package frozen corn 1 8...
3,"['graham cracker crumbs', 'powdered sugar', 'p...",reeses cup candy 1 cup peanut butter 0.75 cup ...
4,"['sour cream', 'bacon', 'pepper', 'extra lean ...",cheeseburger potato soup 6 baking potatoes 1 p...


In [4]:
len(data)

1930209

In [5]:
texts = data['normalized_combined'].tolist()
type(texts)

list

In [15]:
ner_tags = data['NER'].tolist()
ner_tags[0]

"['bite size shredded rice biscuits', 'vanilla', 'brown sugar', 'nuts', 'milk', 'butter']"

In [16]:
# Create a dictionary of terms
terms = {}
patterns = []
nlp = spacy.blank("en")

for tags in ner_tags:
    for tag in tags:
        if tag not in terms and tag!='mix':
            terms[tag] = {'label': 'INGREDIENT'}
            patterns.append(nlp(tag))

# Initialize the PhraseMatcher
ingredient_matcher = PhraseMatcher(nlp.vocab)  # nlp.vocab ~ A storage class for vocabulary and other data shared across a language
ingredient_matcher.add("INGREDIENT", None, *patterns)

In [17]:
nlp.analyze_pipes()

{'summary': {}, 'problems': {}, 'attrs': {}}

In [18]:
@Language.component("ingredient_extractor")
def ingredient_extractor(doc):
    matches = ingredient_matcher(doc)
    spans = [Span(doc, start, end, label='INGREDIENT') for match_id, start, end in matches]

    # Resolve overlaps by keeping the longest span
    filtered_spans = spacy.util.filter_spans(spans)
    
    doc.ents = filtered_spans
    return doc
# Add the custom component to the pipeline
nlp.add_pipe("ingredient_extractor", last=True)

import spacy
from spacy.tokens import Span
from spacy.matcher import Matcher

@Language.component("quantity_extractor")
def quantity_extractor(doc):
    matcher = Matcher(nlp.vocab)
    pattern = [
        {"LIKE_NUM": True},  # Match numbers
        {"LIKE_NUM": True, "OP": "?"},  # Match the second number (e.g., 8), but optional
        {"LOWER": {"IN": ["cup", "tablespoon", "teaspoon", "ounce", "pound", "gram", "kilogram","package", "quart", "liter", "milliliter"]}}
    ]
    matcher.add("QUANTITY", [pattern])
    matches = matcher(doc)
    spans = [Span(doc, start, end, label="QUANTITY") for match_id, start, end in matches]
    filtered_spans = spacy.util.filter_spans(spans)
    
    # Ensure no overlapping entities
    new_ents = [ent for ent in doc.ents if ent.label_ != "QUANTITY"]
    doc.ents = new_ents + filtered_spans
    return doc
nlp.add_pipe("quantity_extractor", last=True)

<function __main__.quantity_extractor(doc)>

In [21]:
nlp.analyze_pipes()

{'summary': {'ingredient_extractor': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'quantity_extractor': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'ingredient_extractor': [], 'quantity_extractor': []},
 'attrs': {}}

In [ ]:
from spacy.tokens import DocBin

train_data = [(text, {"entities": []}) for text in texts]

for i, (text, annotations) in enumerate(train_data):
    doc = nlp(text)
    entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
    train_data[i] = (text, {"entities": entities})
    

In [12]:
#Viewing training data

# Print the first 3 records of train_data
for i in range(5):  # Adjust the range to display more or fewer records
    print(f"Text: {train_data[i][0]}")
    print(f"Entities: {train_data[i][1]}")
    print("-" * 50)  # Separator for readability


Text: no-bake nut cookies 1 cup firmly packed brown sugar 0.5 cup evaporated milk 0.5 teaspoon vanilla 0.5 cup broken nuts pecans 2 tablespoon butter or margarine 3.5 cup bite size shredded rice biscuits in a heavy 2 quart saucepan mix brown sugar nuts evaporated milk and butter or margarine stir over medium heat until mixture bubbles all over top boil and stir 5 minutes more take off heat stir in vanilla and cereal mix well using 2 teaspoon drop and shape into 30 clusters on wax paper let stand until firm about 30 minutes
Entities: {'entities': [(2, 3, 'INGREDIENT'), (20, 25, 'QUANTITY'), (52, 59, 'QUANTITY'), (76, 88, 'QUANTITY'), (97, 104, 'QUANTITY'), (124, 136, 'QUANTITY'), (157, 164, 'QUANTITY'), (201, 202, 'INGREDIENT'), (209, 216, 'QUANTITY'), (429, 439, 'QUANTITY')]}
--------------------------------------------------
Text: jewell ball s chicken 1 small jar chipped beef cut up 4 boned chicken breasts 1 can cream of mushroom soup 1 carton sour cream place chipped beef on bottom 

In [21]:
def save_training_data(data, output_file):
    nlp = spacy.blank("en")
    doc_bin = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annotations["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents
        doc_bin.add(doc)
    doc_bin.to_disk(output_file)

save_training_data(train_data, 'training_data.spacy')

ValueError: [E870] Could not serialize the DocBin because it is too large. Consider splitting up your documents into several doc bins and serializing each separately. spacy.Corpus.v1 will search recursively for all *.spacy files if you provide a directory instead of a filename as the 'path'.

In [ ]:
#Save in batches

import spacy
from spacy.tokens import DocBin

def save_training_data(data, output_dir, chunk_size=5000):
    """Splits data into smaller DocBins and saves each as a separate file."""
    nlp = spacy.blank("en")

    for i in range(0, len(data), chunk_size):
        doc_bin = DocBin()
        batch = data[i:i + chunk_size]  # Get a chunk of data

        for text, annotations in batch:
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in annotations["entities"]:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    ents.append(span)
            doc.ents = ents
            doc_bin.add(doc)

        output_file = f"{output_dir}/training_data_part_{i // chunk_size}.spacy"
        doc_bin.to_disk(output_file)
        print(f" Saved {output_file} with {len(batch)} samples.")

# Save training data in smaller chunks
save_training_data(train_data, 'training_data', chunk_size=5000)


In [ ]:
#Load training data

import glob
import spacy
from spacy.tokens import DocBin

def load_training_data(directory):
    """Loads multiple DocBin files and merges them into a single DocBin."""
    doc_bin = DocBin()  # Create an empty DocBin

    # Load all .spacy files from the directory
    for file in glob.glob(f"{directory}/*.spacy"):
        print(f"📂 Loading {file} ...")
        temp_bin = DocBin().from_disk(file)  # Load individual file
        for doc in temp_bin.get_docs(nlp.vocab):  # Add docs to main DocBin
            doc_bin.add(doc)

    return doc_bin

# Load all training data
nlp = spacy.blank("en")  # Initialize NLP pipeline
db = load_training_data("training_data")

In [ ]:
import random
from spacy.training import Example
from spacy.util import minibatch, compounding

# Load the training data
#nlp = spacy.blank("en")
#db = DocBin().from_disk("training_data.spacy")
docs = list(db.get_docs(nlp.vocab))

# Create the NER component and add it to the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)

# Add the labels to the NER component
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# Disable other pipes during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(5):  # 5 iterations
        random.shuffle(docs)
        losses = {}
        batches = minibatch(docs, size=compounding(4.0, 32.0, 1.5))
        for batch in batches:
            for doc in batch:
                example = Example.from_dict(doc, {"entities": [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]})
                nlp.update([example], drop=0.5, losses=losses)
        print(f"Iteration {itn}, Losses: {losses}")

# Save the trained model to disk
nlp.to_disk("ner_model")

In [ ]:
# Test the custom NER
doc = nlp("Add 2 cups of flour and 1 tablespoon of sugar.")
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
type(train_data)

In [ ]:
def evaluate_model(model, test_data):
    examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in test_data]
    scorer = model.evaluate(examples)
    return scorer

In [ ]:
evaluation_results = evaluate_model(nlp, train_data)
print("Evaluation Results:", evaluation_results)

In [ ]:
# Test the model on some examples
test_texts = "In a heavy 2-quart saucepan, mix 2 lbs brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper.Let stand until firm, about 30 minutes."
doc = nlp(test_texts)

In [ ]:
from spacy import displacy

displacy.render(doc, style='ent')

In [ ]:
real_text = """Peel potatoes and keep aside
Heat oil in a wok
Crackle mustard seeds and urad dal
Add Kari Patta
soute ginger garlic paste and chopped onion
Aad potatoes and soute for 1 to 2minutes
Add all dry masala
Cook with lid on slow flame for 10 minutes
Garnish with chopped coriander
Serve hot with roti or poori
"""

In [ ]:
doc2 = nlp(real_text)

displacy.render(doc2, style='ent')

In [ ]:
import json


entities = []
for ent in doc.ents:
    if ent.label_ == "QUANTITY":
        for i_ent in doc.ents:
            if i_ent.label_ == "INGREDIENT" and i_ent.start == ent.end:
                entities.append({"ingredient": i_ent.text, "quantity": ent.text})
                break
    elif ent.label_ == "INGREDIENT":
        if not any(e["ingredient"] == ent.text for e in entities):
            entities.append({"ingredient": ent.text, "quantity": ""})

entities_json = json.dumps(entities, indent=2)

print(entities_json)